In [ ]:
# load binary file
import numpy as np
import time
from suite2p.run_s2p import run_s2p
import imp


def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'nplanes': 4,
    'diameter': 14,
    'tau': 2.,
    'fs': 2.5,
    'data_path': ['C:/Drive/suite2python/data1/'],
    #'data_path': 'C:\Drive\suite2python\data',
    'subfolders': ['1', '2', '5'],
    'num_workers': 0,
    'reg_tif':False,
    'max_iterations':10,    
      }


# copy tiff to a binary
ops1 = run_s2p(ops)
#ops1 = register.tiff_to_binary(ops)

found ops1 and pre-registered binaries
overwriting ops1 with new ops
skipping registration...
